In [1]:
import drawsvg as draw

In [2]:
class Connector:
    padding = 0.6
    spacing = 0.3
    count = 10
    pad_height = 2.7
    pad_width = 0.7

class Cable:
    length = 19
    width = (
        (Connector.pad_width * Connector.count)
        + (Connector.spacing * (Connector.count - 1))
        + Connector.padding * 2
    )

class SolderPad:
    w = 3
    h = 1
    x_spacing = 10
    y_spacing = 1
    right_start_x = x_spacing / 2 + w / 2
    left_start_x = -right_start_x


In [3]:
def get_drawing():
    width = 16
    height = 30
    d = draw.Drawing(width, height, origin=(-width,0))
    d.svg_args['width'] = f'{width}mm'
    d.svg_args['height'] = f'{height}mm'
    d.svg_args['viewBox'] = f'0 0 {width} {height}'

    d.set_pixel_scale(96 / 25.4)

    return d

In [4]:
def get_layers():    
    edge_layer = draw.Group(
        id="Edge.Cuts", inkscape__groupmode="layer", inkscape__label="Edge.Cuts"
    )
    
    # Objects on the silkscreen layer get traced over as copper pads in KiCad
    back_silkscreen = draw.Group(
        id="B.SilkS", inkscape__groupmode="layer", inkscape__label="B.SilkS"
    )
    
    return edge_layer, back_silkscreen


def edge(solder_pad_height, solder_pad_width):
    p = draw.Path(fill="none", stroke_width=0.1, stroke="black")
    p.M(0, 0)
    p.v(solder_pad_height)
    p.h((solder_pad_width - Cable.width)/2)
    p.v(Cable.length)
    p.h(Cable.width)
    p.v(-Cable.length)
    p.h((solder_pad_width - Cable.width)/2)
    p.v(-solder_pad_height)
    p.Z()
    
    return p

In [5]:
def solder_pads(solder_pad_count, solder_pad_height, solder_pad_width):
    pads = draw.Group()
    params = {'width': SolderPad.w , 'height': SolderPad.h, 'fill': 'black', 'stroke_width': 0}

    for i in range(int(solder_pad_count/2)):
        pads.append(
            draw.Rectangle(
                x=SolderPad.x_spacing / 2,
                y=-(SolderPad.h +  (SolderPad.y_spacing + SolderPad.h) * i),
                **params,
            )
        )

    for i in range(int(solder_pad_count/2)):
        pads.append(
            draw.Rectangle(
                x=-SolderPad. x_spacing / 2 - SolderPad.w ,
                y=-(SolderPad.h + (SolderPad.y_spacing + SolderPad.h) * i),
                **params,
            )
        )
    
    pads.args['transform'] = f'translate({solder_pad_width/2}, {solder_pad_height})'
    return pads

In [6]:
def connection_pads(solder_pad_height, solder_pad_width):
    g = draw.Group()
    for i in range(Connector.count):
        r = draw.Rectangle(
            Connector.padding + (solder_pad_width - Cable.width)/2 + (Connector.pad_width + Connector.spacing)*i,
            solder_pad_height + Cable.length - Connector.pad_height,
            Connector.pad_width,
            Connector.pad_height
        )
        g.append(r)
    
    return g

In [7]:
def cable():
    drawing = get_drawing()
    edge_layer, back_silkscreen = get_layers()
        
    solder_pad_count = Connector.count
    solder_pad_height = (SolderPad.h * Connector.count/2) + SolderPad.h * (Connector.count/2 - 1)
    solder_pad_width = SolderPad.x_spacing + SolderPad.w * 2

    edge_layer.append(edge(solder_pad_height, solder_pad_width))
    drawing.append(edge_layer)
    
    back_silkscreen.append(connection_pads(solder_pad_height, solder_pad_width))
    back_silkscreen.append(solder_pads(solder_pad_count, solder_pad_height, solder_pad_width))
    drawing.append(back_silkscreen)
    
    return drawing

In [8]:
cable().save_svg(f'../outputs/cable_sunshine.svg')